In [1]:
# test version
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version used by PyTorch: {torch.version.cuda}")
print(f"GPU name: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.9.0+cu126
CUDA available: True
CUDA version used by PyTorch: 12.6
GPU name: NVIDIA GeForce RTX 4050 Laptop GPU


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# ---------WORKING WITH DATA--------------
# PyTorch has two primitives to work with data: torch.utils.data.DataLoader and torch.utils.data.Dataset. 
# Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset. 
# The torchvision.datasets module contains Dataset objects for many real-world vison data like CIFAR, COCO.
# Every TorchVision dataset includes 2 arguments: transform and target_transform to modify the samples and labels respectively.



# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
# We pass the Dataset as an argument to DataLoader. this wraps an iterable over our Dataset and supports automatic batching, sampling, 
# shufflling and multiprocess data loading. Here we decide a batch size of 64

batch_size = 64    #each element in the dataloader iterable will return a batch of 64 features and labels.

#create data loaders.
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N,C,H,W]: {X.shape}")
    print(f"Shape of y: {y.shape}")
    break

# Output

# Shape of X [N,C,H,W]: torch.Size([64, 1, 28, 28])
# 64: Batch size - there are 64 images in this batch
# 1: Number of channels - these are grayscale images (1 channel)
# 28: Height - each image is 28 pixels tall
# 28: Width - each image is 28 pixels wide

# Shape of y: torch.Size([64]) 
# 64 labels for each of 64 images in the batch.

Shape of X [N,C,H,W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])


In [ ]:
#-------------------CREATING MODELS----------------------
# To define a neral network in Pytorch, we create a class that inherits from nn.Module.
# We define the layers of the network in the __init__ function and specify how data will pass through the network in the forward function.

# Use torch.accelerator to accelerator operations in the neural network.
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):                        # NeuralNetwork class inherits from nn.Module ,nn.Module is PyTorch's base class for all network components.
    def __init__(self):                                # [64, 1, 28, 28] → [64, 784] (64 batches, 784 features)
        super().__init__()
        self.flatten = nn.Flatten()                    # Create a flattening layer that converts 2D to 1D vectors, needed because linear layers expect 1D input. 
        self.linear_relu_stack = nn.Sequential(        # nn.Sequential()): Container that chains multiple layers together, execute in order from input to output
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    

model = NeuralNetwork().to(device)
print(model)


Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [13]:
#--------------------OPTIMIZE THE MODEL PARAMETERS-------------------------
# To train a model, we need a loss function and an optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)

# In a single training loop, the model makes predictions on the training dataset (fed to it in batches), 
# and backpropagates the prediction error to adjust the model's parameters.

def train(dataloader, model, loss_func, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X, y= X.to(device), y.to(device)

        # compute prediction error
        pred = model(X)
        loss = loss_func(pred,y)

        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch  + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


In [15]:
# Check model's performence to ensure it is learning
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [17]:
epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_func, optimizer)
    test(test_dataloader, model, loss_func)
print("Done!")

Epoch 1
-------------------------------
loss: 1.161928 [   64/60000]
loss: 1.158481 [ 6464/60000]
loss: 0.978176 [12864/60000]
loss: 1.121189 [19264/60000]
loss: 0.984356 [25664/60000]
loss: 1.021807 [32064/60000]
loss: 1.050056 [38464/60000]
loss: 0.985201 [44864/60000]
loss: 1.042055 [51264/60000]
loss: 0.960111 [57664/60000]
Test Error: 
 Accuracy: 65.5%, Avg loss: 0.982296 

Epoch 2
-------------------------------
loss: 1.042994 [   64/60000]
loss: 1.058550 [ 6464/60000]
loss: 0.862283 [12864/60000]
loss: 1.028735 [19264/60000]
loss: 0.894240 [25664/60000]
loss: 0.926581 [32064/60000]
loss: 0.973912 [38464/60000]
loss: 0.909596 [44864/60000]
loss: 0.961536 [51264/60000]
loss: 0.894464 [57664/60000]
Test Error: 
 Accuracy: 66.9%, Avg loss: 0.909716 

Epoch 3
-------------------------------
loss: 0.956917 [   64/60000]
loss: 0.990426 [ 6464/60000]
loss: 0.780415 [12864/60000]
loss: 0.963603 [19264/60000]
loss: 0.834470 [25664/60000]
loss: 0.857575 [32064/60000]
loss: 0.921135 [38464/

In [18]:
# Saving Model
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [19]:
# Loading Model
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [24]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]

with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
